# Spark DataFrame
* Tabular data
  * Rows 
  * Named Columns
* Immutable 
* Distribute collection of data
* Lazy
* Can process structured and semi-structured data
  * relational database
  * csv
  * json
  * txt
  * RDD
  * dict
  * list
  * etc
* Support SQL or expression methods
  * SELECT * FROM RedWine
  * red_wine_df.select()
* Schema
  * Information about
    * column name
    * data type
    * empty values
    * etc
  * Help to optimize the queries

In [1]:
from pyspark import SparkContext, SparkConf

from pyspark.sql import SparkSession

master = 'spark://192.168.2.102:7077' # Connect to remote server
appName = 'Create DataFrame'

conf = SparkConf()\
    .setMaster(master)\
    .setAppName(appName)\
    .set("spark.executor.memory", "2g")\
    .set("spark.cores.max", "4")\

# RDD
sc = SparkContext(conf=conf)

# DataFrame
spark = SparkSession.builder.getOrCreate()

21/12/22 16:46:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/22 16:46:37 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
df = spark.createDataFrame([
    {'Id': 1, 'Value': 1},
    {'Id': 1, 'Value': 2},
    {'Id': 2, 'Value': 3},
    {'Id': 2, 'Value': 4},
])
df.show()

+---+-----+
| Id|Value|
+---+-----+
|  1|    1|
|  1|    2|
|  2|    3|
|  2|    4|
+---+-----+



In [3]:
df = spark.createDataFrame([
    [1,1],
    [1,2],
    [2,3],
    [2,4],
], schema=['Id', 'Value'])
df.show()

+---+-----+
| Id|Value|
+---+-----+
|  1|    1|
|  1|    2|
|  2|    3|
|  2|    4|
+---+-----+




### Create DataFrame from RDD

In [4]:
red_wine_rdd = sc.parallelize([
    [7.4, 0.7, 0.0, 5],
    [7.8, 0.88, 0.0, 5],
    [7.8, 0.76, 0.04, 5],
    [11.2, 0.28, 0.56, 6],
])

red_wine_rdd.take(5)

[[7.4, 0.7, 0.0, 5],
 [7.8, 0.88, 0.0, 5],
 [7.8, 0.76, 0.04, 5],
 [11.2, 0.28, 0.56, 6]]

In [5]:
columns = ['fixed acidity', 'volatile acidity', 'citric acid', 'quality']

red_wine_df = spark.createDataFrame(red_wine_rdd, schema=columns)

red_wine_df.show()

+-------------+----------------+-----------+-------+
|fixed acidity|volatile acidity|citric acid|quality|
+-------------+----------------+-----------+-------+
|          7.4|             0.7|        0.0|      5|
|          7.8|            0.88|        0.0|      5|
|          7.8|            0.76|       0.04|      5|
|         11.2|            0.28|       0.56|      6|
+-------------+----------------+-----------+-------+



In [6]:
red_wine_df.dtypes

[('fixed acidity', 'double'),
 ('volatile acidity', 'double'),
 ('citric acid', 'double'),
 ('quality', 'bigint')]

### Create DataFrame from csv

In [7]:
root_path = 'hdfs://192.168.2.102:9000/dataset/{filename}'

# By Default inferSchema is False
red_wine_df = spark.read.csv(root_path.format(filename='winequality-red.csv'), header=True, inferSchema=False)

red_wine_df.show(5)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|          7.4|             0.7|        0.0|           1.9|    0.076|               11.0|                34.0| 0.9978|3.51|     0.56|    9.4|      5|
|          7.8|            0.88|        0.0|           2.6|    0.098|               25.0|                67.0| 0.9968| 3.2|     0.68|    9.8|      5|
|          7.8|            0.76|       0.04|           2.3|    0.092|               15.0|                54.0|  0.997|3.26|     0.65|    9.8|      5|
|         11.2|            0.28|       0.56|           1.9|    0.075|               17.0|           

In [8]:
red_wine_df.dtypes

[('fixed acidity', 'string'),
 ('volatile acidity', 'string'),
 ('citric acid', 'string'),
 ('residual sugar', 'string'),
 ('chlorides', 'string'),
 ('free sulfur dioxide', 'string'),
 ('total sulfur dioxide', 'string'),
 ('density', 'string'),
 ('pH', 'string'),
 ('sulphates', 'string'),
 ('alcohol', 'string'),
 ('quality', 'string')]

### Create DataFrame from txt

In [9]:
temp_hist = spark.read.text('data/beer_temp_hist.txt')
temp_hist.show(5)

+-----------------+
|            value|
+-----------------+
|2021-12-01;1;20.0|
|2021-12-02;1;20.2|
|    2021-12-03;1;|
|2021-12-04;1;20.3|
|2021-12-05;1;20.5|
+-----------------+
only showing top 5 rows



In [10]:
rdd = sc.textFile('data/beer_temp_hist.txt')
rdd.take(5)

['2021-12-01;1;20.0',
 '2021-12-02;1;20.2',
 '2021-12-03;1;',
 '2021-12-04;1;20.3',
 '2021-12-05;1;20.5']

In [11]:
splitted_rows = rdd.map(lambda row: row.split(';'))
temp_hist = spark.createDataFrame(splitted_rows, schema=['Date', 'BeerId', 'Temp'])
temp_hist.show()

+----------+------+----+
|      Date|BeerId|Temp|
+----------+------+----+
|2021-12-01|     1|20.0|
|2021-12-02|     1|20.2|
|2021-12-03|     1|    |
|2021-12-04|     1|20.3|
|2021-12-05|     1|20.5|
|2021-12-01|     2|16.5|
|2021-12-02|     2|16.4|
|2021-12-03|     2|16.5|
|2021-12-04|     2|    |
|2021-12-05|     2|16.8|
|2021-12-05|     2|16.7|
|2021-12-01|     3|18.3|
|2021-12-02|     3|18.4|
|2021-12-03|     3|    |
|2021-12-01|     4|18.2|
+----------+------+----+



In [12]:
temp_hist.dtypes

[('Date', 'string'), ('BeerId', 'string'), ('Temp', 'string')]

### Infer data type

In [13]:
red_wine_df = spark.read.csv(root_path.format(filename='winequality-red.csv'), header=True, inferSchema=True)
red_wine_df.dtypes

[('fixed acidity', 'double'),
 ('volatile acidity', 'double'),
 ('citric acid', 'double'),
 ('residual sugar', 'double'),
 ('chlorides', 'double'),
 ('free sulfur dioxide', 'double'),
 ('total sulfur dioxide', 'double'),
 ('density', 'double'),
 ('pH', 'double'),
 ('sulphates', 'double'),
 ('alcohol', 'double'),
 ('quality', 'int')]

### Schema

In [14]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import IntegerType, FloatType, DateType, StringType

In [15]:
temp_hist_schema = StructType([
    StructField('Date', DateType()),
    StructField('BeerId', IntegerType()),
    StructField('Temp', FloatType()),
])

In [16]:

temp_hist_df = spark.read.csv('data/beer_temp_hist.txt', sep=';', schema=temp_hist_schema)
temp_hist_df.show()

+----------+------+----+
|      Date|BeerId|Temp|
+----------+------+----+
|2021-12-01|     1|20.0|
|2021-12-02|     1|20.2|
|2021-12-03|     1|null|
|2021-12-04|     1|20.3|
|2021-12-05|     1|20.5|
|2021-12-01|     2|16.5|
|2021-12-02|     2|16.4|
|2021-12-03|     2|16.5|
|2021-12-04|     2|null|
|2021-12-05|     2|16.8|
|2021-12-05|     2|16.7|
|2021-12-01|     3|18.3|
|2021-12-02|     3|18.4|
|2021-12-03|     3|null|
|2021-12-01|     4|18.2|
+----------+------+----+



In [17]:
temp_hist_df.dtypes

[('Date', 'date'), ('BeerId', 'int'), ('Temp', 'float')]

In [18]:
temp_hist_df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- BeerId: integer (nullable = true)
 |-- Temp: float (nullable = true)

